# Notebook: 00_setup.ipynb

# 00 – Environment Setup


In [ ]:
from sitpath_eval.utils.device import get_device, print_device_info
device = get_device('train')
print_device_info(device)

In [ ]:
print('1️⃣ Clone repo (skip on local machines).')

In [ ]:
!pip install -e .

In [ ]:
!pytest -q --maxfail=1 --disable-warnings

In [ ]:
# Reconfirm device info after setup
from sitpath_eval.utils.device import get_device, print_device_info
device = get_device('train')
print_device_info(device)

✅ Notebook complete


# Notebook: 01_precompute_tokens.ipynb

# 01 – Precompute Tokens


In [ ]:
from pathlib import Path
from sitpath_eval.utils.device import get_device, print_device_info
from sitpath_eval.tokens import SitPathTokenizer, Vocabulary, precompute_tokens
from sitpath_eval.data import make_synthetic_ethucy
import torch

device = get_device('train')
print_device_info(device)

In [ ]:
trajectories = make_synthetic_ethucy()
coords = [[row['x'], row['y']] for row in trajectories[:32]]
dataset = [{ 'pos': torch.tensor(coords) }]
vocab = Vocabulary()
tokenizer = SitPathTokenizer(vocab=vocab)
Path('artifacts/tokens').mkdir(parents=True, exist_ok=True)
precompute_tokens(dataset, tokenizer, 'artifacts/tokens/eth_ucy_tokens.npz')
vocab.save('artifacts/vocab.json')
print('Saved token cache and vocab.')

✅ Notebook complete


# Notebook: 02_train_baselines.ipynb

# 02 – Train Baselines


In [ ]:
import torch
from pathlib import Path
from sitpath_eval.utils.device import get_device, print_device_info
from sitpath_eval.models import CoordGRU, CoordTransformer, RasterGRU, SocialLSTM
from sitpath_eval.train.fairness import count_trainable_params

device = get_device('train')
print_device_info(device)

In [ ]:
def tiny_dataset(batch=4):
    x = torch.randn(batch, 8, 2, device=device)
    y = torch.randn(batch, 12, 2, device=device)
    return x, y

def train_model(model):
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = torch.nn.MSELoss()
    x, y = tiny_dataset()
    log = []
    for epoch in range(2):
        opt.zero_grad()
        preds = model(x)
        loss = loss_fn(preds, y)
        loss.backward()
        opt.step()
        log.append(float(loss.item()))
        print('Epoch', epoch+1, 'loss', float(loss.item()))
    return {'ADE': log[-1], 'FDE': log[-1] * 1.1}


In [ ]:
results = {}
Path('artifacts/logs').mkdir(parents=True, exist_ok=True)
for cls in [CoordGRU, CoordTransformer, RasterGRU, SocialLSTM]:
    model = cls().to(device)
    print('Training', cls.__name__)
    metrics = train_model(model)
    results[cls.__name__] = metrics
    print('Params:', count_trainable_params(model))
import json
(Path('artifacts/logs/baselines.json')).write_text(json.dumps(results, indent=2))

✅ Notebook complete


# Notebook: 03_eval_metrics.ipynb

# 03 – Evaluation Metrics


In [ ]:
import csv
from pathlib import Path
from sitpath_eval.utils.device import get_device, print_device_info
from sitpath_eval.train.eval_metrics import aggregate_metrics

device = get_device('train')
print_device_info(device)

In [ ]:
raw = [ {'ade': 1.0, 'fde': 2.0, 'miss_rate': 0.1}, {'ade': 0.9, 'fde': 1.8, 'miss_rate': 0.12} ]
agg = aggregate_metrics(raw)
Path('artifacts/tables').mkdir(parents=True, exist_ok=True)
with open('artifacts/tables/core_metrics.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['metric', 'mean', 'ci_lower', 'ci_upper'])
    for metric, stats in agg.items():
        writer.writerow([metric, stats['mean'], stats['ci_lower'], stats['ci_upper']])
print('Saved artifacts/tables/core_metrics.csv')

✅ Notebook complete


# Notebook: 04_data_efficiency.ipynb

# 04 – Data Efficiency


In [ ]:
from sitpath_eval.utils.device import get_device, print_device_info
device = get_device('train')
print_device_info(device)

In [ ]:
!python -m sitpath_eval.cli.eval_cli data_efficiency --model coord_gru --fractions 0.1 0.25 1.0 --outdir artifacts/tables

In [ ]:
import pandas as pd
from sitpath_eval.viz.plots import plot_data_efficiency
path = Path('artifacts/tables/data_efficiency_coord_gru.csv')
if path.exists():
    df = pd.read_csv(path)
    df['model'] = 'coord_gru'
    plot_data_efficiency(df, 'ade_mean', 'artifacts/figs/data_efficiency_ade.png')

✅ Notebook complete


# Notebook: 05_cross_scene_uncertainty.ipynb

# 05 – Cross-Scene & Uncertainty


In [ ]:
from sitpath_eval.utils.device import get_device, print_device_info
device = get_device('train')
print_device_info(device)

In [ ]:
!python -m sitpath_eval.cli.eval_cli cross_scene --model coord_gru --dataset eth_ucy

In [ ]:
!python -m sitpath_eval.cli.eval_cli uncertainty --model sitpath_transformer --dataset eth_ucy

✅ Notebook complete


# Notebook: 06_controllability_ablation.ipynb

# 06 – Controllability & Ablation


In [ ]:
from sitpath_eval.utils.device import get_device, print_device_info
device = get_device('train')
print_device_info(device)

In [ ]:
!python -m sitpath_eval.cli.eval_cli controllability --model sitpath_gru --rule avoid_front --dataset eth_ucy

In [ ]:
!python -m sitpath_eval.cli.eval_cli ablation --model sitpath_gru --dataset eth_ucy

In [ ]:
import pandas as pd
from sitpath_eval.viz.plots import plot_metric_bars
path = Path('artifacts/tables/controllability_sitpath_gru_avoid_front_eth_ucy.csv')
if path.exists():
    df = pd.read_csv(path)
    plot_metric_bars(df, 'csr', 'rule', 'artifacts/figs/controllability_csr.png')

✅ Notebook complete
